# 04 · Perfilado de clusters e insights accionables


In [1]:
# ============================================================
# NB-04 · PASO 0 — CARGA Y PREPARACIÓN MÍNIMA
# ============================================================

# 1) Imports y configuración base (mismo patrón que NB-02/NB-03)
import sys                                 # manipular sys.path si hace falta
from pathlib import Path                    # manejo robusto de rutas
from dotenv import load_dotenv              # leer variables desde .env
import os                                   # acceder a variables de entorno
import pandas as pd                         # dataframes
import numpy as np                          # utilidades numéricas

# Definimos ROOT como la carpeta raíz del proyecto (sube un nivel desde /notebooks)
ROOT = Path.cwd().parent                    # asume que estás dentro de /notebooks
if str(ROOT) not in sys.path:               # agrega ROOT al sys.path si no está
    sys.path.append(str(ROOT))

load_dotenv()                               # carga variables del archivo .env (si existe)

# Opciones de impresión para no truncar columnas
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

# 2) Rutas de entrada (con defaults y posibilidad de override vía .env)
DEFAULT_BASE = ROOT / "data" / "processed" / "activos_for_model_v2.csv"                 # features baseline
DEFAULT_IDS_C = ROOT / "data" / "processed" / "activos_ids_v2_plus_clustered.csv"      # IDs + clusters

X_BASE_PATH = Path(os.getenv("X_BASE_PATH", str(DEFAULT_BASE)))                         # permite override
X_IDS_CLUSTERED_PATH = Path(os.getenv("X_IDS_CLUSTERED_PATH", str(DEFAULT_IDS_C)))      # permite override

# 3) Función de lectura segura (misma idea que usamos antes)
def _leer_csv_seguro(path: Path, nombre_log: str) -> pd.DataFrame:
    """Lee un CSV validando existencia y devolviendo error claro si falla."""
    if not path.exists():
        raise FileNotFoundError(f"[{nombre_log}] No se encontró el archivo: {path}")
    try:
        return pd.read_csv(path, low_memory=False)
    except Exception as e:
        raise RuntimeError(f"[{nombre_log}] Error al leer {path}:\n{e}")

# 4) Carga efectiva
df_base = _leer_csv_seguro(X_BASE_PATH, "BASELINE FEATURES")                 # matriz baseline (sin Empresa)
df_ids_c = _leer_csv_seguro(X_IDS_CLUSTERED_PATH, "IDS + CLUSTERS")          # IDs con labels (baseline y con Empresa)

# 5) Echo de rutas y shapes
print("=== RUTAS USADAS ===")
print("Features baseline :", X_BASE_PATH)
print("IDs + clusters    :", X_IDS_CLUSTERED_PATH)
print("\n=== SHAPES ===")
print("df_base :", df_base.shape)   # filas x columnas de features
print("df_ids_c:", df_ids_c.shape)  # filas x columnas de IDs + clusters

# 6) Chequeo de columnas de cluster esperadas en df_ids_c
cols_cluster_esperadas = {"cluster_baseline", "cluster_con_empresa"}
presentes = set(c for c in df_ids_c.columns if c.startswith("cluster_"))
print("\n=== Etiquetas detectadas en df_ids_c ===")
print(sorted(list(presentes)))
faltan = cols_cluster_esperadas - presentes
if faltan:
    print(f"⚠️ Faltan columnas de cluster: {faltan}. Revisa NB-03 export.")
else:
    print("OK: están cluster_baseline y cluster_con_empresa.")

# 7) Intento de key para merge (preferimos RutBeneficiario si está)
#    Nota: df_base (features) NO debería contener PII; muchas veces no trae IDs.
#    Usaremos merge si hubiera una key común. Si no, alinearemos por índice.
merge_key = None
for key_candidata in ["RutBeneficiario", "rut_beneficiario", "rut", "ID", "id"]:
    if key_candidata in df_base.columns and key_candidata in df_ids_c.columns:
        merge_key = key_candidata
        break

# 8) Unimos etiquetas al dataset de features
if merge_key:
    # Caso 1: hay key común → merge explícito
    df_feat = df_base.merge(df_ids_c[[merge_key, "cluster_baseline", "cluster_con_empresa"]],
                            on=merge_key, how="left", validate="m:1")
    print(f"\nUnión por key '{merge_key}' realizada.")
else:
    # Caso 2: sin key común → alineación por índice (asumimos orden idéntico de NB-02→NB-03)
    #         Esta es la vía habitual si la matriz de features no incluye IDs por privacidad.
    if len(df_base) != len(df_ids_c):
        raise ValueError("No hay key común y el nº de filas difiere entre df_base y df_ids_c.")
    df_feat = df_base.copy()
    df_feat["cluster_baseline"] = df_ids_c["cluster_baseline"].values
    df_feat["cluster_con_empresa"] = df_ids_c["cluster_con_empresa"].values
    print("\nUnión por índice realizada (mismo orden de filas).")

print("Shape df_feat (features + clusters):", df_feat.shape)

# 9) Detección de grupos de columnas (para perfilar en siguientes pasos)
#    a) Presencias
PRES_COLS = [c for c in df_feat.columns if c.endswith("_pres")]  # Atencion15d_pres, etc.
#    b) Numéricas clave (mismo set de NB-03, pero usamos las presentes)
NUMERIC_CORE_CANON = [
    "Edad","CantPptos","CantPptosAbo","CantPptosAvan",
    "TotPptos_l1p","TotPptosAbo_l1p","TotPptosAvan_l1p",
    "PctCumplimiento","TicketPromPpto_l1p"
]
NUM_COLS_PRESENTES = [c for c in NUMERIC_CORE_CANON if c in df_feat.columns]

#    c) Dummies de geografía (si están en baseline)
DUM_COMUNA = [c for c in df_feat.columns if c.startswith("Comuna_grp_")]
DUM_REGION = [c for c in df_feat.columns if c.startswith("Region_")]

# 10) Echo rápido de lo detectado
print("\n=== Columnas detectadas ===")
print("Presencias (", len(PRES_COLS), "):", PRES_COLS)
print("Numéricas  (", len(NUM_COLS_PRESENTES), "):", NUM_COLS_PRESENTES)
print("Dummies Comuna (", len(DUM_COMUNA), ")")
print("Dummies Región (", len(DUM_REGION), ")")

# 11) Mini-checks
print("\n=== MINI-CHECKS ===")
print("¿clusters presentes?               ", "OK ✅" if {"cluster_baseline","cluster_con_empresa"}.issubset(df_feat.columns) else "Revisar ⚠️")
print("¿sin NaNs en columnas de cluster?  ", "OK ✅" if df_feat[["cluster_baseline","cluster_con_empresa"]].isna().sum().sum()==0 else "Revisar ⚠️")
print("¿presencias detectadas?            ", "OK ✅" if len(PRES_COLS)>0 else "No hay presencias ⚠️")
print("¿numéricas clave detectadas?       ", "OK ✅" if len(NUM_COLS_PRESENTES)>0 else "Faltan numéricas ⚠️")

# (Opcional) Vista rápida de tamaños por cluster (baseline)
if "cluster_baseline" in df_feat.columns:
    print("\nTamaño por cluster (baseline):")
    print(df_feat["cluster_baseline"].value_counts().sort_index())


=== RUTAS USADAS ===
Features baseline : /Users/santiagotupper/Documents/DATA ANALYSIS/Portal Orto Data/analisis-portal-ortodoncia/data/processed/activos_for_model_v2.csv
IDs + clusters    : /Users/santiagotupper/Documents/DATA ANALYSIS/Portal Orto Data/analisis-portal-ortodoncia/data/processed/activos_ids_v2_plus_clustered.csv

=== SHAPES ===
df_base : (14141, 48)
df_ids_c: (14141, 6)

=== Etiquetas detectadas en df_ids_c ===
['cluster_baseline', 'cluster_con_empresa']
OK: están cluster_baseline y cluster_con_empresa.

Unión por índice realizada (mismo orden de filas).
Shape df_feat (features + clusters): (14141, 50)

=== Columnas detectadas ===
Presencias ( 4 ): ['Atencion15d_pres', 'Atencion1m_pres', 'Atencion3m_pres', 'Atencion6m_pres']
Numéricas  ( 9 ): ['Edad', 'CantPptos', 'CantPptosAbo', 'CantPptosAvan', 'TotPptos_l1p', 'TotPptosAbo_l1p', 'TotPptosAvan_l1p', 'PctCumplimiento', 'TicketPromPpto_l1p']
Dummies Comuna ( 21 )
Dummies Región ( 14 )

=== MINI-CHECKS ===
¿clusters prese

In [2]:
# ============================================================
# NB-04 · PASO 1 — PERFIL GENERAL POR CLUSTER (BASELINE)
# ============================================================

# 1) Definimos el nombre de la columna de cluster a usar (baseline)
CLUSTER_COL = "cluster_baseline"  # elegimos la variante baseline como estándar

# 2) Validación suave: asegurarnos de que la columna de cluster está en df_feat
assert CLUSTER_COL in df_feat.columns, "cluster_baseline no está en df_feat. Revisa el Paso 0."

# 3) (Opcional) Forzamos el tipo de dato del cluster a entero para ordenar bien
df_feat[CLUSTER_COL] = df_feat[CLUSTER_COL].astype(int)  # nos aseguramos que el cluster sea int (0,1,2,...)

# 4) Guardamos una copia sólo con columnas relevantes para evitar confusiones
cols_para_perfil = [CLUSTER_COL] + NUM_COLS_PRESENTES + PRES_COLS  # combinamos cluster + numéricas + presencias
dfp = df_feat[cols_para_perfil].copy()                              # subset limpio para perfilar

# 5) Calculamos el "n" (número de pacientes) por cluster para contexto
n_por_cluster = dfp.groupby(CLUSTER_COL).size()                     # cuenta filas por cluster
n_por_cluster.name = "n_pacientes"                                  # renombramos la serie para que tenga nombre claro

# 6) Calculamos MEDIANAS de las numéricas por cluster (robusto a outliers)
medianas_num = (
    dfp.groupby(CLUSTER_COL)[NUM_COLS_PRESENTES]    # agrupamos por cluster y seleccionamos numéricas
       .median(numeric_only=True)                   # calculamos mediana por columna
       .add_suffix("_mediana")                      # añadimos sufijo para distinguir después
)

# 7) Calculamos MEDIAS de las numéricas por cluster (útil como contraste de la mediana)
medias_num = (
    dfp.groupby(CLUSTER_COL)[NUM_COLS_PRESENTES]    # agrupamos por cluster
       .mean(numeric_only=True)                     # calculamos media por columna
       .add_suffix("_media")                        # sufijo para diferenciar
)

# 8) Calculamos TASAS (proporciones) de presencias por cluster
tasas_pres = (
    dfp.groupby(CLUSTER_COL)[PRES_COLS]             # agrupamos por cluster y tomamos presencias 0/1
       .mean(numeric_only=True)                     # la media de 0/1 es la proporción
       .add_suffix("_tasa")                         # sufijo para identificar como tasa
)

# 9) Unimos todo en un único DataFrame de perfil
perfil_general = (
    pd.concat([n_por_cluster, medianas_num, medias_num, tasas_pres], axis=1)  # concatenamos por columnas
      .sort_index()                                                           # ordenamos por índice de cluster (0,1,2,…)
      .round(3)                                                               # redondeamos a 3 decimales para lectura
)

# 10) Reordenamos columnas para que primero vaya el "n", luego numéricas y al final presencias
#     (esto es opcional, sólo mejora legibilidad)
cols_ordenadas = (
    ["n_pacientes"] +                                   # primero el tamaño del cluster
    sorted([c for c in perfil_general.columns if c.endswith("_mediana")]) +  # luego medianas
    sorted([c for c in perfil_general.columns if c.endswith("_media")]) +    # luego medias
    sorted([c for c in perfil_general.columns if c.endswith("_tasa")])       # y al final tasas de presencia
)
perfil_general = perfil_general[cols_ordenadas]          # aplicamos el nuevo orden de columnas

# 11) Mostramos el perfil completo (técnico) para revisión
print("=== PERFIL GENERAL (técnico) — Baseline ===")
display(perfil_general)

# 12) Construimos una versión COMPACTA “para gerente” con 4–6 KPIs clave
#     Elegimos: Edad_mediana, CantPptos_mediana, TotPptos_l1p_mediana, PctCumplimiento_mediana,
#               Atencion6m_pres_tasa (y opcionalmente Atencion3m_pres_tasa)
cols_ejecutivas = []
# 12.1) Añadimos columnas si existen (evita fallos si faltara alguna)
for c in [
    "Edad_mediana",
    "CantPptos_mediana",
    "TotPptos_l1p_mediana",
    "PctCumplimiento_mediana",
    "Atencion6m_pres_tasa",
    "Atencion3m_pres_tasa"
]:
    if c in perfil_general.columns:
        cols_ejecutivas.append(c)                       # sólo agregamos las que existan en este corte

# 13) Creamos la tabla ejecutiva con n_pacientes + KPIs seleccionados
perfil_ejecutivo = (
    perfil_general[["n_pacientes"] + cols_ejecutivas]   # seleccionamos columnas ejecutivas
      .copy()                                           # copiamos para manipular sin side-effects
)

# 14) (Opcional) Convertimos tasas (0–1) a porcentaje simple para lectura rápida
for c in perfil_ejecutivo.columns:
    if c.endswith("_tasa"):
        perfil_ejecutivo[c] = (perfil_ejecutivo[c] * 100).round(1)  # ejemplo: 0.44 → 44.0 (%)

# 15) Renombramos columnas a algo “más humano” para gerente
renombres = {
    "n_pacientes": "Pacientes",
    "Edad_mediana": "Edad (mediana)",
    "CantPptos_mediana": "Presupuestos (mediana)",
    "TotPptos_l1p_mediana": "Monto total (log1p, mediana)",
    "PctCumplimiento_mediana": "% Cumplimiento (mediana)",
    "Atencion6m_pres_tasa": "Presencia 6m (%)",
    "Atencion3m_pres_tasa": "Presencia 3m (%)"
}
perfil_ejecutivo = perfil_ejecutivo.rename(columns=renombres)  # aplicamos nombres legibles

# 16) Mostramos la tabla ejecutiva
print("\n=== PERFIL EJECUTIVO (para gerente) — Baseline ===")
display(perfil_ejecutivo)

# 17) (Opcional) Exportamos a CSV para reportes/notion
OUTPUT_DIR = ROOT / "reports"                                 # carpeta de reports
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)                 # creamos si no existe
perfil_general.to_csv(OUTPUT_DIR / "nb04_perfil_general_baseline.csv")     # export técnico
perfil_ejecutivo.to_csv(OUTPUT_DIR / "nb04_perfil_ejecutivo_baseline.csv") # export ejecutivo

print("\nArchivos guardados en /reports/:")
print(" - nb04_perfil_general_baseline.csv")
print(" - nb04_perfil_ejecutivo_baseline.csv")


=== PERFIL GENERAL (técnico) — Baseline ===


,n_pacientes,CantPptosAbo_mediana,CantPptosAvan_mediana,CantPptos_mediana,Edad_mediana,PctCumplimiento_mediana,TicketPromPpto_l1p_mediana,TotPptosAbo_l1p_mediana,TotPptosAvan_l1p_mediana,TotPptos_l1p_mediana,CantPptosAbo_media,CantPptosAvan_media,CantPptos_media,Edad_media,PctCumplimiento_media,TicketPromPpto_l1p_media,TotPptosAbo_l1p_media,TotPptosAvan_l1p_media,TotPptos_l1p_media,Atencion15d_pres_tasa,Atencion1m_pres_tasa,Atencion3m_pres_tasa,Atencion6m_pres_tasa
cluster_baseline,,,,,,,,,,,,,,,,,,,,,,,
0,10613,1.0,1.0,1.0,35.0,0.667,12.773,9.909,9.893,13.315,0.979,0.974,1.698,37.046,0.629,12.666,8.598,8.554,13.079,0.066,0.134,0.263,0.402
1,3216,4.0,4.0,5.0,26.0,0.800,12.546,13.920,13.911,14.203,4.297,4.285,5.582,29.192,0.801,12.551,13.820,13.809,14.202,0.149,0.304,0.489,0.598
2,312,0.0,0.0,1.0,38.0,0.000,0.000,0.000,0.000,0.000,0.006,0.000,1.048,39.952,0.002,0.000,0.069,0.000,0.000,0.035,0.083,0.199,0.298



=== PERFIL EJECUTIVO (para gerente) — Baseline ===


,Pacientes,Edad (mediana),Presupuestos (mediana),"Monto total (log1p, mediana)",% Cumplimiento (mediana),Presencia 6m (%),Presencia 3m (%)
cluster_baseline,,,,,,,
0,10613,35.0,1.0,13.315,0.667,40.2,26.3
1,3216,26.0,5.0,14.203,0.800,59.8,48.9
2,312,38.0,1.0,0.000,0.000,29.8,19.9



Archivos guardados en /reports/:
 - nb04_perfil_general_baseline.csv
 - nb04_perfil_ejecutivo_baseline.csv


In [3]:
# ============================================================
# NB-04 · PASO 2 — GEOGRAFÍA POR CLUSTER
# ============================================================

# 1) Definimos la columna de cluster a usar (baseline)
CLUSTER_COL = "cluster_baseline"

# 2) Validamos que tenemos dummies de Región y Comuna
assert len(DUM_REGION) > 0, "No se detectaron columnas de Región en df_feat."
assert len(DUM_COMUNA) > 0, "No se detectaron columnas de Comuna en df_feat."

# 3) Reconstruimos nombres legibles a partir de las columnas dummies
#    Ejemplo: 'Region_Metropolitana' -> 'Metropolitana'
REGION_LABELS = [c.replace("Region_", "") for c in DUM_REGION]
COMUNA_LABELS = [c.replace("Comuna_grp_", "") for c in DUM_COMUNA]

# 4) Creamos DataFrames auxiliares con clusters + dummies
df_region = df_feat[[CLUSTER_COL] + DUM_REGION].copy()   # dataset con regiones
df_comuna = df_feat[[CLUSTER_COL] + DUM_COMUNA].copy()   # dataset con comunas

# 5) Calculamos distribución por REGIÓN (% pacientes en cada región por cluster)
dist_region = (
    df_region.groupby(CLUSTER_COL)[DUM_REGION]     # agrupamos por cluster
             .mean()                               # media de dummies = proporción
             .round(3)                             # redondeamos a 3 decimales
)
dist_region.columns = REGION_LABELS                # renombramos columnas a etiquetas legibles

# 6) Calculamos distribución por COMUNA (% pacientes en cada comuna por cluster)
dist_comuna = (
    df_comuna.groupby(CLUSTER_COL)[DUM_COMUNA]
             .mean()
             .round(3)
)
dist_comuna.columns = COMUNA_LABELS

# 7) Extraemos resumen para Región Metropolitana vs. resto
if "Metropolitana" in dist_region.columns:
    rm_vs_resto = (
        dist_region[["Metropolitana"]]                           # columna de RM
        .assign(Otras=1 - dist_region["Metropolitana"])          # calculamos complemento = otras regiones
        .round(3)
    )
else:
    rm_vs_resto = None
    print("⚠️ No se encontró columna de Región Metropolitana en los datos.")

# 8) Para comunas: ordenamos y mostramos el Top-N (ej. top 10) por cluster
topN = 10
top_comunas = {}
for cluster in dist_comuna.index:                    # iteramos por cluster
    serie = dist_comuna.loc[cluster].sort_values(ascending=False)  # ordenamos comunas por % en ese cluster
    top_comunas[cluster] = serie.head(topN)          # guardamos top N
    print(f"\n=== TOP {topN} COMUNAS — Cluster {cluster} ===")
    print(top_comunas[cluster])

# 9) Mostramos tablas resumen
print("\n=== DISTRIBUCIÓN POR REGIÓN (porcentaje) ===")
display(dist_region * 100)    # multiplicamos por 100 para leer como %
if rm_vs_resto is not None:
    print("\n=== REGIÓN METROPOLITANA vs OTRAS ===")
    display(rm_vs_resto * 100)

print("\n=== DISTRIBUCIÓN POR COMUNA (primeras columnas, %) ===")
display((dist_comuna * 100).iloc[:, :10])   # mostramos solo primeras 10 columnas para no saturar

# 10) Exportamos resultados a reports/
dist_region.to_csv(ROOT / "reports" / "nb04_dist_region_baseline.csv")
dist_comuna.to_csv(ROOT / "reports" / "nb04_dist_comuna_baseline.csv")
if rm_vs_resto is not None:
    rm_vs_resto.to_csv(ROOT / "reports" / "nb04_rm_vs_resto_baseline.csv")

print("\nArchivos guardados en /reports/:")
print(" - nb04_dist_region_baseline.csv")
print(" - nb04_dist_comuna_baseline.csv")
if rm_vs_resto is not None:
    print(" - nb04_rm_vs_resto_baseline.csv")


⚠️ No se encontró columna de Región Metropolitana en los datos.

=== TOP 10 COMUNAS — Cluster 0 ===
Otras/Infreq        0.265
Santiago            0.158
Estación Central    0.060
Pudahuel            0.056
Maipú               0.053
Cerro Navia         0.038
Quinta Normal       0.037
Puente Alto         0.036
La Florida          0.034
Providencia         0.033
Name: 0, dtype: float64

=== TOP 10 COMUNAS — Cluster 1 ===
Otras/Infreq        0.221
Santiago            0.177
Pudahuel            0.062
Maipú               0.059
Estación Central    0.053
Cerro Navia         0.050
Quinta Normal       0.044
Renca               0.040
Puente Alto         0.036
Quilicura           0.034
Name: 1, dtype: float64

=== TOP 10 COMUNAS — Cluster 2 ===
Sin Comuna          0.295
Otras/Infreq        0.173
Santiago            0.163
Iquique             0.090
Pudahuel            0.054
Puente Alto         0.042
Estación Central    0.026
Ñuñoa               0.022
Maipú               0.022
San Bernardo        0.016


,Antofagasta,Atacama,Coquimbo,Región Metropolitana de Santiago,Región de Los Lagos,Región de Los Ríos,Región de Magallanes y de la Antártica Chilena,Región de la Araucanía,Región del Biobío,Región del Libertador Gral. Bernardo O’Higgins,Región del Maule,Sin Región,Tarapacá,Valparaíso
cluster_baseline,,,,,,,,,,,,,,
0,0.1,0.0,0.1,97.5,0.0,0.0,0.0,0.0,0.1,0.3,0.0,0.9,0.2,0.6
1,0.0,0.0,0.1,98.3,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.2,1.0,0.4
2,0.0,0.0,0.0,60.9,0.0,0.0,0.0,0.0,0.0,0.6,0.0,29.5,9.0,0.0



=== DISTRIBUCIÓN POR COMUNA (primeras columnas, %) ===


,Cerro Navia,Conchalí,Estación Central,Iquique,La Florida,Lo Prado,Maipú,Otras/Infreq,Pedro Aguirre Cerda,Peñalolén
cluster_baseline,,,,,,,,,,
0,3.8,1.7,6.0,0.2,3.4,2.8,5.3,26.5,1.8,2.1
1,5.0,2.3,5.3,1.0,2.7,3.0,5.9,22.1,2.0,1.4
2,1.3,1.0,2.6,9.0,1.0,1.3,2.2,17.3,0.0,1.3



Archivos guardados en /reports/:
 - nb04_dist_region_baseline.csv
 - nb04_dist_comuna_baseline.csv


In [4]:
# ============================================================
# NB-04 · PASO 3 — EMPRESA/CONVENIO POR CLUSTER
# ============================================================

# 1) Definimos el cluster a usar (baseline como referencia)
CLUSTER_COL = "cluster_baseline"

# 2) Rutas al dataset con Empresa/Convenio
DEFAULT_EMP = ROOT / "data" / "processed" / "activos_for_model_v2_empresa.csv"
X_EMP_PATH = Path(os.getenv("X_EMP_PATH", str(DEFAULT_EMP)))

# 3) Leemos el dataset con Empresa/Convenio
df_emp = pd.read_csv(X_EMP_PATH, low_memory=False)

# 4) Verificamos shape y columnas relevantes
print("Shape df_emp:", df_emp.shape)
print("Columnas disponibles (primeras 20):", df_emp.columns[:20].tolist())

# 5) Detectamos columnas de interés
cols_empresa = [c for c in df_emp.columns if c.startswith("Empresa_grp_")]   # dummies de empresas top
cols_binarias = [c for c in ["TieneEmpresa","EsConvenio"] if c in df_emp.columns]  # flags binarios

print("Cols binarios:", cols_binarias)
print("Cols dummies Empresa:", cols_empresa[:10], "...")

# 6) Alineamos con etiquetas de cluster
#    Asumimos mismo orden que df_ids_c (como en paso 0)
if len(df_emp) != len(df_ids_c):
    raise ValueError("El nº de filas de df_emp y df_ids_c no coincide. Revisa NB-02/03 exports.")

df_emp_feat = df_emp.copy()
df_emp_feat["cluster_baseline"] = df_ids_c["cluster_baseline"].values
df_emp_feat["cluster_con_empresa"] = df_ids_c["cluster_con_empresa"].values

print("Shape df_emp_feat:", df_emp_feat.shape)

# 7) Calculamos proporción de binarios por cluster
res_binarios = (
    df_emp_feat.groupby(CLUSTER_COL)[cols_binarias]
               .mean()
               .round(3)
)
print("\n=== PROPORCIÓN BINARIOS POR CLUSTER ===")
display(res_binarios * 100)   # en porcentaje

# 8) Calculamos top empresas por cluster (dummies)
top_empresas = {}
for cluster in df_emp_feat[CLUSTER_COL].unique():
    serie = df_emp_feat.loc[df_emp_feat[CLUSTER_COL]==cluster, cols_empresa].mean().sort_values(ascending=False)
    top_empresas[cluster] = serie.head(5).round(3)   # top 5 por cluster
    print(f"\n=== TOP 5 EMPRESAS — Cluster {cluster} ===")
    print((top_empresas[cluster] * 100).astype(str) + " %")

# 9) Exportamos resultados
res_binarios.to_csv(ROOT / "reports" / "nb04_empresa_binarios_baseline.csv")
for cluster, serie in top_empresas.items():
    serie.to_csv(ROOT / "reports" / f"nb04_top_empresas_cluster{cluster}.csv")

print("\nArchivos guardados en /reports/:")
print(" - nb04_empresa_binarios_baseline.csv")
print(" - nb04_top_empresas_cluster*.csv (uno por cluster)")


Shape df_emp: (14141, 67)
Columnas disponibles (primeras 20): ['Edad', 'CantPptos', 'CantPptosAbo', 'CantPptosAvan', 'TotPptos_l1p', 'TotPptosAbo_l1p', 'TotPptosAvan_l1p', 'PctCumplimiento', 'TicketPromPpto_l1p', 'Atencion15d_pres', 'Atencion1m_pres', 'Atencion3m_pres', 'Atencion6m_pres', 'Comuna_grp_Cerro Navia', 'Comuna_grp_Conchalí', 'Comuna_grp_Estación Central', 'Comuna_grp_Iquique', 'Comuna_grp_La Florida', 'Comuna_grp_Lo Prado', 'Comuna_grp_Maipú']
Cols binarios: ['TieneEmpresa', 'EsConvenio']
Cols dummies Empresa: ['Empresa_grp_BANCO DE CHILE', 'Empresa_grp_CAJA 18 DE SEPTIEMBRE TRABAJADORES', 'Empresa_grp_CAJA COMPENSACION LOS ANDES', 'Empresa_grp_CAJA LOS HEROES', 'Empresa_grp_CORP.IP. ESC. CONTADORES Y AUDITORE', 'Empresa_grp_DUOC UC', 'Empresa_grp_FONASA', 'Empresa_grp_HOSPITAL DR. LUIS CALVO MACKENNA', 'Empresa_grp_INST CHILENO BRITANICO DE CULTURA', 'Empresa_grp_ISAPRE'] ...
Shape df_emp_feat: (14141, 69)

=== PROPORCIÓN BINARIOS POR CLUSTER ===


,TieneEmpresa,EsConvenio
cluster_baseline,,
0,99.7,0.0
1,98.9,0.0
2,89.7,0.0



=== TOP 5 EMPRESAS — Cluster 2 ===
Empresa_grp_FONASA                                 39.4 %
Empresa_grp_KEEPSMILING CHILE SPA    34.599999999999994 %
Empresa_grp_SIN EMPRESA              10.299999999999999 %
Empresa_grp_ISAPRE                    5.800000000000001 %
Empresa_grp_CAJA LOS HEROES                         4.8 %
dtype: object

=== TOP 5 EMPRESAS — Cluster 0 ===
Empresa_grp_FONASA                         75.5 %
Empresa_grp_ISAPRE                          9.2 %
Empresa_grp_CAJA COMPENSACION LOS ANDES     6.0 %
Empresa_grp_CAJA LOS HEROES                 2.6 %
Empresa_grp_OTRAS/INFREQ                    1.9 %
dtype: object

=== TOP 5 EMPRESAS — Cluster 1 ===
Empresa_grp_FONASA                                   68.7 %
Empresa_grp_ISAPRE                                   25.5 %
Empresa_grp_TARJETA SOY PROVIDENCIA                   1.3 %
Empresa_grp_SIN EMPRESA                1.0999999999999999 %
Empresa_grp_OTRAS/INFREQ               1.0999999999999999 %
dtype: object

Archivos 

In [5]:
# ============================================================
# NB-04 · PASO 4 — INSIGHTS EJECUTIVOS POR CLUSTER
# ============================================================

from datetime import date   # para poner fecha en el reporte

# 1) Definimos la ruta de salida
OUTPUT_MD = ROOT / "reports" / "nb04_insights_executive.md"

# 2) Creamos un diccionario con los insights clave por cluster
insights = {
    0: {
        "perfil": "Adultos (mediana 35 años), 1 presupuesto típico, cumplimiento ≈67%, 40% con visita en 6m.",
        "geografia": "Dispersos en muchas comunas de RM (Santiago, Est. Central, Maipú, Pudahuel).",
        "empresa": "Predominio Fonasa (75%), baja participación Isapres.",
        "accion": [
            "Campañas de reactivación (recordatorios, chequeos preventivos).",
            "Enfoque en Fonasa/cobertura básica.",
            "Mensajes amplios en comunas populosas de la RM."
        ]
    },
    1: {
        "perfil": "Jóvenes (mediana 26 años), 5 presupuestos, cumplimiento ≈80%, 60% con visita en 6m.",
        "geografia": "Altamente concentrados en RM (98%). Fuerte en Santiago centro, Maipú, Pudahuel.",
        "empresa": "Mayor peso de Isapres (25%), aunque mayoría aún Fonasa.",
        "accion": [
            "Programas de fidelización (planes de mantención, beneficios exclusivos).",
            "Campañas de referidos (jóvenes atraen a pares).",
            "Ofertas premium enfocadas en Isapres."
        ]
    },
    2: {
        "perfil": "Adultos (mediana 38 años), casi sin presupuestos ni pagos, cumplimiento 0%, solo 30% con visita en 6m.",
        "geografia": "60% en RM, 9% en Tarapacá/Iquique, 30% con región/comuna faltante.",
        "empresa": "Registros atípicos (empresa específica 34%, 10% sin empresa).",
        "accion": [
            "Revisión de calidad de datos (depurar registros incompletos).",
            "No priorizar campañas específicas (bajo retorno esperado)."
        ]
    }
}

# 3) Construimos el contenido del archivo Markdown
lines = []
lines.append(f"# Insights ejecutivos por cluster (NB-04)\n")
lines.append(f"Fecha de generación: {date.today().isoformat()}\n")
lines.append("---\n")

for cluster, info in insights.items():
    lines.append(f"## Cluster {cluster}\n")
    lines.append(f"- **Perfil:** {info['perfil']}")
    lines.append(f"- **Geografía:** {info['geografia']}")
    lines.append(f"- **Empresa/Convenio:** {info['empresa']}")
    lines.append("- **Acciones recomendadas:**")
    for acc in info["accion"]:
        lines.append(f"  - {acc}")
    lines.append("")   # línea en blanco

# 4) Escribimos el archivo a disco
OUTPUT_MD.parent.mkdir(parents=True, exist_ok=True)   # creamos carpeta reports/ si no existe
with open(OUTPUT_MD, "w", encoding="utf-8") as f:
    f.write("\n".join(lines))

print(f"✅ Reporte generado en: {OUTPUT_MD}")


✅ Reporte generado en: /Users/santiagotupper/Documents/DATA ANALYSIS/Portal Orto Data/analisis-portal-ortodoncia/reports/nb04_insights_executive.md


In [6]:
# ============================================================
# NB-04 · Exportar comparativa: Dashboard vs Clustering (CSV)
# y mensaje corto para WhatsApp al gerente
# ============================================================

# 1) Imports mínimos (pandas para el CSV, Path para rutas)
import pandas as pd                 # manejo de DataFrames y export a CSV
from pathlib import Path            # manejo robusto de rutas (independiente de SO)
import os                           # por si necesitas leer variables de entorno

# 2) Aseguramos la raíz del proyecto (ROOT) como en NB-02/NB-04 previos
try:
    ROOT  # si ya existe ROOT en el notebook, no hacemos nada
except NameError:
    ROOT = Path.cwd().parent        # si no existe, lo definimos asumiendo que estás en /notebooks

# 3) Carpeta de salida de reportes
REPORTS_DIR = ROOT / "reports"      # ruta de la carpeta reports/
REPORTS_DIR.mkdir(parents=True, exist_ok=True)  # crea la carpeta si no existe

# 4) Armamos la tabla comparativa como lista de dicts (más legible y explícito)
rows = [
    {
        "Metrica": "Pacientes activos",
        "Dashboard (≈jul/2025)": "14500 aprox",
        "Clustering (jun/2025)": "14141",
        "Comentario / Valor agregado": "La diferencia se explica porque el corte de datos es un mes anterior"
    },
    {
        "Metrica": "% con atención en 6m",
        "Dashboard (≈jul/2025)": "40–45%",
        "Clustering (jun/2025)": "Cluster 0: 40% | Cluster 1: 60% | Cluster 2: 30%",
        "Comentario / Valor agregado": "El promedio ponderado ≈43%, consistente con dashboard. Se identifica qué grupos sostienen esas tasas"
    },
    {
        "Metrica": "% Cumplimiento",
        "Dashboard (≈jul/2025)": "≈70% global",
        "Clustering (jun/2025)": "Cluster 0: 67% | Cluster 1: 80% | Cluster 2: 0%",
        "Comentario / Valor agregado": "El promedio ponderado ≈70%. Se distingue quiénes cumplen más (jóvenes del Cluster 1)"
    },
    {
        "Metrica": "Distribución geográfica (RM)",
        "Dashboard (≈jul/2025)": "≈95% RM / 5% otras",
        "Clustering (jun/2025)": "Cluster 0: 97,5% RM | Cluster 1: 98,3% RM | Cluster 2: 61% RM",
        "Comentario / Valor agregado": "Consistente con el global. El cluster 2 explica la dispersión fuera de RM"
    },
    {
        "Metrica": "Empresa/Convenio",
        "Dashboard (≈jul/2025)": "Fonasa mayoritario / Isapres 20–25%",
        "Clustering (jun/2025)": "Cluster 0: Fonasa 75%, Isapres 9% | Cluster 1: Fonasa 69%, Isapres 26% | Cluster 2: mezcla atípica",
        "Comentario / Valor agregado": "Consistente en global. Se revela perfil socioeconómico por cluster"
    },
]

# 5) Convertimos a DataFrame
df_comp = pd.DataFrame(rows)

# 6) Guardamos a CSV (utf-8-sig para compatibilidad con Excel)
csv_path = REPORTS_DIR / "nb04_dashboard_vs_clustering.csv"
df_comp.to_csv(csv_path, index=False, encoding="utf-8-sig")

# 7) Mostramos ruta de salida y preview
print("✅ CSV generado en:", csv_path)
print("\nVista previa:")
display(df_comp)

# 8) Mensaje breve para WhatsApp (impreso para copiar/pegar)
whatsapp_msg = (
    "Hola [Nombre],\n"
    "te comparto una tabla comparativa entre los números del dashboard y lo que obtuve con el análisis de clustering "
    "(con corte un mes antes). Los promedios son consistentes con el tablero, pero el valor agregado está en que ahora "
    "puedo distinguir qué grupos de pacientes explican esos promedios y qué acciones se recomiendan para cada uno.\n"
    "Te adjunto el CSV resumido."
)
print("\n--- Mensaje para WhatsApp ---\n")
print(whatsapp_msg)


✅ CSV generado en: /Users/santiagotupper/Documents/DATA ANALYSIS/Portal Orto Data/analisis-portal-ortodoncia/reports/nb04_dashboard_vs_clustering.csv

Vista previa:


,Metrica,Dashboard (≈jul/2025),Clustering (jun/2025),Comentario / Valor agregado
0,Pacientes activos,14500 aprox,14141,La diferencia se explica porque el corte de da...
1,% con atención en 6m,40–45%,Cluster 0: 40% | Cluster 1: 60% | Cluster 2: 30%,"El promedio ponderado ≈43%, consistente con da..."
2,% Cumplimiento,≈70% global,Cluster 0: 67% | Cluster 1: 80% | Cluster 2: 0%,El promedio ponderado ≈70%. Se distingue quién...
3,Distribución geográfica (RM),≈95% RM / 5% otras,"Cluster 0: 97,5% RM | Cluster 1: 98,3% RM | Cl...",Consistente con el global. El cluster 2 explic...
4,Empresa/Convenio,Fonasa mayoritario / Isapres 20–25%,"Cluster 0: Fonasa 75%, Isapres 9% | Cluster 1:...",Consistente en global. Se revela perfil socioe...



--- Mensaje para WhatsApp ---

Hola [Nombre],
te comparto una tabla comparativa entre los números del dashboard y lo que obtuve con el análisis de clustering (con corte un mes antes). Los promedios son consistentes con el tablero, pero el valor agregado está en que ahora puedo distinguir qué grupos de pacientes explican esos promedios y qué acciones se recomiendan para cada uno.
Te adjunto el CSV resumido.


In [3]:
# ============================================================
# NB-04 · Exportar insights ejecutivos (Markdown → PDF)
# - Ruta A: PDF directo con pdfkit + wkhtmltopdf (si está instalado)
# - Ruta B: HTML estilado listo para "Imprimir → Guardar como PDF"
# ============================================================

# 1) Imports necesarios
from pathlib import Path          # manejo robusto de rutas
import shutil                     # para detectar binarios instalados (wkhtmltopdf)
import sys                        # para mensajes de error claros
import textwrap                   # para formatear CSS embebido
import datetime as dt             # para sellar fecha en el HTML
try:
    import markdown               # conversión Markdown → HTML (pip install markdown)
except ImportError as e:
    raise SystemExit("Falta la librería 'markdown'. Instala con: pip install markdown") from e

# 2) Definimos ROOT igual que en el resto del proyecto
try:
    ROOT  # si ya existe (de pasos anteriores), lo reutilizamos
except NameError:
    ROOT = Path.cwd().parent      # asumimos que estamos en /notebooks

# 3) Entradas y salidas
md_path   = ROOT / "reports" / "nb04_insights_executive.md"  # archivo Markdown de NB-04
html_path = ROOT / "reports" / "nb04_insights_executive.html" # HTML intermedio y/o final
pdf_path  = ROOT / "reports" / "nb04_insights_executive.pdf"  # PDF destino

# 4) Comprobamos que el .md existe
if not md_path.exists():
    raise FileNotFoundError(f"No se encontró el archivo Markdown en: {md_path}\n"
                            f"Genera primero el .md con el Paso 4 de NB-04.")

# 5) Leemos el contenido del Markdown
md_text = md_path.read_text(encoding="utf-8")

# 6) Convertimos Markdown a HTML (cuerpo) usando 'markdown'
#    Agregamos extensiones útiles (tablas, listas, etc.)
html_body = markdown.markdown(
    md_text,
    extensions=["extra", "toc", "sane_lists", "smarty"]
)

# 7) Preparamos un HTML completo con estilo básico y tipografía
#    (Esto hace que se vea profesional al abrirlo en el navegador/imprimir)
today = dt.date.today().strftime("%Y-%m-%d")
css = textwrap.dedent("""
    /* Estilos simples para lectura e impresión */
    @media print {
        body { -webkit-print-color-adjust: exact; }
    }
    body {
        font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", Roboto, Inter, Arial, sans-serif;
        line-height: 1.45;
        color: #1a1a1a;
        margin: 40px auto;
        max-width: 860px;
        padding: 0 16px;
        background: #ffffff;
    }
    h1, h2, h3 {
        margin-top: 1.2em;
        margin-bottom: 0.6em;
        line-height: 1.2;
    }
    h1 { font-size: 1.9rem; }
    h2 { font-size: 1.4rem; border-bottom: 1px solid #eee; padding-bottom: 0.3rem; }
    h3 { font-size: 1.15rem; }
    p, li { font-size: 0.98rem; }
    code, pre { background: #f6f8fa; border-radius: 6px; padding: 2px 5px; }
    blockquote {
        border-left: 4px solid #ddd; margin: 1em 0; padding: 0.5em 1em; color: #555;
        background: #fafafa;
    }
    table {
        border-collapse: collapse; width: 100%; margin: 1em 0; font-size: 0.95rem;
    }
    th, td {
        border: 1px solid #e6e6e6; padding: 8px 10px; text-align: left;
    }
    th { background: #f2f2f2; }
    .meta {
        font-size: 0.85rem; color: #666; margin-bottom: 1rem;
    }
""").strip()

# 8) Envolvemos el cuerpo HTML en una plantilla completa
html_full = f"""<!doctype html>
<html lang="es">
<head>
<meta charset="utf-8">
<meta name="viewport" content="width=device-width,initial-scale=1" />
<title>Insights ejecutivos por cluster (NB-04)</title>
<style>{css}</style>
</head>
<body>
<div class="meta">Generado el {today} · Archivo fuente: {md_path.name}</div>
{html_body}
</body>
</html>
"""

# 9) Guardamos el HTML estilado (sirve como respaldo o para imprimir manualmente a PDF)
html_path.write_text(html_full, encoding="utf-8")
print("✅ HTML generado:", html_path)

# 10) Intentamos la Ruta A: PDF directo con pdfkit + wkhtmltopdf
#     - Requiere: pip install pdfkit  y tener instalado el binario 'wkhtmltopdf'
#     - Si no están, caemos a la Ruta B (usar HTML y 'Guardar como PDF')
try:
    import pdfkit  # conversor HTML → PDF que usa wkhtmltopdf
    wkhtml_bin = shutil.which("wkhtmltopdf")  # buscamos el binario en el sistema
    if wkhtml_bin is None:
        raise FileNotFoundError("wkhtmltopdf no está instalado en el sistema.")
    # Configuramos pdfkit para usar el binario detectado
    config = pdfkit.configuration(wkhtmltopdf=wkhtml_bin)
    # Opciones para márgenes y calidad (puedes ajustarlas)
    options = {
        "page-size": "A4",
        "margin-top": "12mm",
        "margin-right": "12mm",
        "margin-bottom": "14mm",
        "margin-left": "12mm",
        "encoding": "UTF-8",
        "print-media-type": None,
        "enable-local-file-access": None
    }
    # Generamos el PDF
    pdfkit.from_file(str(html_path), str(pdf_path), configuration=config, options=options)
    print("✅ PDF generado:", pdf_path)
    print("\nPuedes adjuntarlo por WhatsApp sin problemas.")
except Exception as e:
    # 11) Ruta B (fallback): no se pudo crear PDF directo
    #     Explicamos claramente cómo usar el HTML para "Imprimir → Guardar como PDF"
    print("\n⚠️ No se pudo generar PDF de forma automática.")
    print("Motivo:", e)
    print("\nPlan B (universal):")
    print(f"1) Abre el HTML en tu navegador: {html_path}")
    print("2) Menú Imprimir → Destino: Guardar como PDF → Guardar.")
    print("   (Queda un PDF igual de presentable para enviar por WhatsApp.)")


✅ HTML generado: /Users/santiagotupper/Documents/DATA ANALYSIS/Portal Orto Data/analisis-portal-ortodoncia/reports/nb04_insights_executive.html

⚠️ No se pudo generar PDF de forma automática.
Motivo: wkhtmltopdf no está instalado en el sistema.

Plan B (universal):
1) Abre el HTML en tu navegador: /Users/santiagotupper/Documents/DATA ANALYSIS/Portal Orto Data/analisis-portal-ortodoncia/reports/nb04_insights_executive.html
2) Menú Imprimir → Destino: Guardar como PDF → Guardar.
   (Queda un PDF igual de presentable para enviar por WhatsApp.)
